# LSTM-DCF Training Data Pipeline Analysis

**Research Reference**: [LSTM Networks for Estimating Growth Rates in DCF Models](https://www.revocm.com/articles/lstm-networks-estimating-growth-rates-dcf-models)

## Step 1: Import Libraries & Setup Paths

In [1]:
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path().absolute().parent if Path().absolute().name == 'notebooks' else Path().absolute()
sys.path.insert(0, str(PROJECT_ROOT))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import yfinance as yf
from datetime import datetime

# Project imports
from src.data.processors.time_series_processor import TimeSeriesProcessor
from src.data.fetchers import YFinanceFetcher
from src.models.deep_learning.lstm_dcf import LSTMDCFModel
from config.settings import RAW_DATA_DIR, PROCESSED_DATA_DIR, MODELS_DIR

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully")
print(f"Project Root: {PROJECT_ROOT}")

✓ Libraries imported successfully
Project Root: c:\Users\Hans8899\Desktop\fyp\emetix


## Step 2: Check Raw Financial Statement Files

We have financial statement CSVs stored in `data/raw/financial_statements/`. Let's verify what we have.

In [2]:
# Check what raw financial statement files exist
financial_statements_dir = RAW_DATA_DIR / 'financial_statements'

print("=" * 80)
print("📁 RAW FINANCIAL STATEMENT FILES")
print("=" * 80)
print(f"\nDirectory: {financial_statements_dir}")
print(f"Exists: {financial_statements_dir.exists()}\n")

if financial_statements_dir.exists():
    files = list(financial_statements_dir.glob('*.csv'))
    print(f"✅ Found {len(files)} CSV files")
    
    # Count by type
    income_files = [f for f in files if '_income' in f.name]
    cashflow_files = [f for f in files if '_cashflow' in f.name]
    balance_files = [f for f in files if '_balance' in f.name]
    
    print(f"\n📊 File breakdown:")
    print(f"   Income Statements: {len(income_files)}")
    print(f"   Cash Flow Statements: {len(cashflow_files)}")
    print(f"   Balance Sheets: {len(balance_files)}")
    
    # Get unique tickers
    tickers_with_financials = set([f.name.split('_')[0] for f in files])
    print(f"\n🏢 Unique tickers with financial data: {len(tickers_with_financials)}")
    print(f"   Sample tickers: {list(sorted(tickers_with_financials))[:10]}")
else:
    print("❌ Directory not found!")

📁 RAW FINANCIAL STATEMENT FILES

Directory: C:\Users\Hans8899\Desktop\fyp\emetix\data\raw\financial_statements
Exists: True

✅ Found 355 CSV files

📊 File breakdown:
   Income Statements: 120
   Cash Flow Statements: 118
   Balance Sheets: 117

🏢 Unique tickers with financial data: 120
   Sample tickers: ['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADP', 'AMAT', 'AMD', 'AMGN']


In [3]:
# Load and examine AAPL financial statements
ticker = 'AAPL'

print("=" * 80)
print(f"🔍 EXAMINING {ticker} FINANCIAL STATEMENTS")
print("=" * 80)

# Load income statement
income_path = financial_statements_dir / f'{ticker}_income.csv'
cashflow_path = financial_statements_dir / f'{ticker}_cashflow.csv'
balance_path = financial_statements_dir / f'{ticker}_balance.csv'

if income_path.exists():
    income_df = pd.read_csv(income_path)
    print(f"\n✅ Income Statement: {income_df.shape}")
    print(f"   Columns: {list(income_df.columns)}")
    print(f"\n   📊 KEY COLUMNS WE NEED:")
    key_cols = ['totalRevenue', 'operatingIncome', 'netIncome', 'depreciationAndAmortization']
    for col in key_cols:
        has_col = col in income_df.columns
        print(f"      {'✅' if has_col else '❌'} {col}")
    
    # Show sample data
    print(f"\n   Sample Revenue data (last 5 years):")
    if 'totalRevenue' in income_df.columns:
        print(income_df[['fiscalDateEnding', 'totalRevenue']].head().to_string(index=False))

if cashflow_path.exists():
    cashflow_df = pd.read_csv(cashflow_path)
    print(f"\n✅ Cash Flow Statement: {cashflow_df.shape}")
    print(f"   Columns: {list(cashflow_df.columns)}")
    print(f"\n   📊 KEY COLUMNS WE NEED:")
    key_cols = ['operatingCashflow', 'capitalExpenditures', 'depreciationDepletionAndAmortization']
    for col in key_cols:
        has_col = col in cashflow_df.columns
        print(f"      {'✅' if has_col else '❌'} {col}")
    
    # Show sample data
    print(f"\n   Sample CapEx data (last 5 quarters):")
    if 'capitalExpenditures' in cashflow_df.columns:
        print(cashflow_df[['fiscalDateEnding', 'operatingCashflow', 'capitalExpenditures']].head().to_string(index=False))

if balance_path.exists():
    balance_df = pd.read_csv(balance_path)
    print(f"\n✅ Balance Sheet: {balance_df.shape}")
    print(f"   Key column: totalAssets {'✅' if 'totalAssets' in balance_df.columns else '❌'}")

🔍 EXAMINING AAPL FINANCIAL STATEMENTS

✅ Income Statement: (81, 26)
   Columns: ['fiscalDateEnding', 'reportedCurrency', 'grossProfit', 'totalRevenue', 'costOfRevenue', 'costofGoodsAndServicesSold', 'operatingIncome', 'sellingGeneralAndAdministrative', 'researchAndDevelopment', 'operatingExpenses', 'investmentIncomeNet', 'netInterestIncome', 'interestIncome', 'interestExpense', 'nonInterestIncome', 'otherNonOperatingIncome', 'depreciation', 'depreciationAndAmortization', 'incomeBeforeTax', 'incomeTaxExpense', 'interestAndDebtExpense', 'netIncomeFromContinuingOperations', 'comprehensiveIncomeNetOfTax', 'ebit', 'ebitda', 'netIncome']

   📊 KEY COLUMNS WE NEED:
      ✅ totalRevenue
      ✅ operatingIncome
      ✅ netIncome
      ✅ depreciationAndAmortization

   Sample Revenue data (last 5 years):
fiscalDateEnding  totalRevenue
      2025-06-30   94036000000
      2025-03-31   95359000000
      2024-12-31  124300000000
      2024-09-30   94930000000
      2024-06-30   85777000000

✅ Cash 

In [4]:
# Quick column check to see what's actually available
print("🔍 QUICK COLUMN CHECK")
print("=" * 80)

ticker = 'AAPL'
income_path = financial_statements_dir / f'{ticker}_income.csv'
cashflow_path = financial_statements_dir / f'{ticker}_cashflow.csv'

# Check if files exist first
if income_path.exists() and cashflow_path.exists():
    income_cols = pd.read_csv(income_path, nrows=0).columns.tolist()
    cashflow_cols = pd.read_csv(cashflow_path, nrows=0).columns.tolist()
    
    print(f"\n📊 Income Statement Columns ({len(income_cols)}):")
    print(f"   {income_cols[:15]}")
    
    print(f"\n📊 Cashflow Statement Columns ({len(cashflow_cols)}):")
    print(f"   {cashflow_cols[:15]}")
    
    # Check for key columns
    print(f"\n✅ Key columns present:")
    key_checks = {
        'totalRevenue': 'totalRevenue' in income_cols,
        'operatingIncome': 'operatingIncome' in income_cols,
        'depreciationAndAmortization': 'depreciationAndAmortization' in income_cols,
        'ebitda': 'ebitda' in income_cols,
        'netIncome': 'netIncome' in income_cols,
        'operatingCashflow': 'operatingCashflow' in cashflow_cols,
        'capitalExpenditures': 'capitalExpenditures' in cashflow_cols
    }
    
    for col, present in key_checks.items():
        print(f"   {col:30s}: {'✅ YES' if present else '❌ NO'}")
else:
    print("❌ Files not found!")

🔍 QUICK COLUMN CHECK

📊 Income Statement Columns (26):
   ['fiscalDateEnding', 'reportedCurrency', 'grossProfit', 'totalRevenue', 'costOfRevenue', 'costofGoodsAndServicesSold', 'operatingIncome', 'sellingGeneralAndAdministrative', 'researchAndDevelopment', 'operatingExpenses', 'investmentIncomeNet', 'netInterestIncome', 'interestIncome', 'interestExpense', 'nonInterestIncome']

📊 Cashflow Statement Columns (29):
   ['fiscalDateEnding', 'reportedCurrency', 'operatingCashflow', 'paymentsForOperatingActivities', 'proceedsFromOperatingActivities', 'changeInOperatingLiabilities', 'changeInOperatingAssets', 'depreciationDepletionAndAmortization', 'capitalExpenditures', 'changeInReceivables', 'changeInInventory', 'profitLoss', 'cashflowFromInvestment', 'cashflowFromFinancing', 'proceedsFromRepaymentsOfShortTermDebt']

✅ Key columns present:
   totalRevenue                  : ✅ YES
   operatingIncome               : ✅ YES
   depreciationAndAmortization   : ✅ YES
   ebitda                      

## Step 4: Build Proper Training Dataset with Margins & Growth Rates

Create the enhanced dataset combining function that includes:
- Core DCF metrics (Revenue, CapEx, D&A, FCF)
- Profitability margins (Operating, Net, FCF, EBITDA)
- Growth rates (target variables)
- Normalization by Total Assets

In [5]:
# ============ QUICK VALIDATION: SINGLE STOCK PREVIEW ============
print("=" * 100)
print("🔍 QUICK DATASET VALIDATION (AAPL Only)")
print("=" * 100)

ticker = 'AAPL'

try:
    # Load financial statements
    income_df = pd.read_csv(financial_statements_dir / f'{ticker}_income.csv')
    cashflow_df = pd.read_csv(financial_statements_dir / f'{ticker}_cashflow.csv')
    balance_df = pd.read_csv(financial_statements_dir / f'{ticker}_balance.csv')
    
    print(f"\n✅ Loaded {ticker} financial statements")
    print(f"   Income: {income_df.shape[0]} quarters")
    print(f"   Cashflow: {cashflow_df.shape[0]} quarters")
    print(f"   Balance: {balance_df.shape[0]} quarters")
    
    # Extract core columns
    income_subset = income_df[['fiscalDateEnding', 'totalRevenue', 'operatingIncome', 
                                 'depreciationAndAmortization', 'ebitda', 'netIncome']].copy()
    cashflow_subset = cashflow_df[['fiscalDateEnding', 'operatingCashflow', 'capitalExpenditures']].copy()
    balance_subset = balance_df[['fiscalDateEnding', 'totalAssets', 'totalLiabilities']].copy()
    
    # Merge
    df = income_subset.merge(cashflow_subset, on='fiscalDateEnding', how='inner') \
                      .merge(balance_subset, on='fiscalDateEnding', how='inner')
    
    df['date'] = pd.to_datetime(df['fiscalDateEnding'])
    df = df.sort_values('date').reset_index(drop=True)
    
    # Convert to numeric
    df['revenue'] = pd.to_numeric(df['totalRevenue'], errors='coerce')
    df['operating_cf'] = pd.to_numeric(df['operatingCashflow'], errors='coerce')
    df['capex'] = pd.to_numeric(df['capitalExpenditures'], errors='coerce').abs()
    df['da'] = pd.to_numeric(df['depreciationAndAmortization'], errors='coerce')
    df['total_assets'] = pd.to_numeric(df['totalAssets'], errors='coerce')
    df['net_income'] = pd.to_numeric(df['netIncome'], errors='coerce')
    df['ebitda'] = pd.to_numeric(df['ebitda'], errors='coerce')
    df['operating_income'] = pd.to_numeric(df['operatingIncome'], errors='coerce')
    
    # Calculate FCF
    df['fcf'] = df['operating_cf'] - df['capex']
    
    # Margins
    df['fcf_margin'] = (df['fcf'] / df['revenue']) * 100
    df['operating_margin'] = (df['operating_income'] / df['revenue']) * 100
    df['net_margin'] = (df['net_income'] / df['revenue']) * 100
    
    # Normalize by assets
    df['revenue_per_asset'] = df['revenue'] / df['total_assets']
    df['fcf_per_asset'] = df['fcf'] / df['total_assets']
    
    # Growth rates (TARGET VARIABLES)
    df['revenue_growth'] = df['revenue'].pct_change() * 100
    df['fcf_growth'] = df['fcf'].pct_change() * 100
    
    print(f"\n✅ COMBINED DATASET")
    print(f"   Shape: {df.shape}")
    print(f"   Date range: {df['date'].min().date()} to {df['date'].max().date()}")
    
    print(f"\n📊 SAMPLE: Last 5 Quarters")
    display_cols = ['date', 'revenue', 'fcf', 'revenue_growth', 'fcf_growth', 'fcf_margin', 'revenue_per_asset']
    sample = df[display_cols].tail(5).copy()
    sample['revenue_B'] = sample['revenue'] / 1e9
    sample['fcf_B'] = sample['fcf'] / 1e9
    print(sample[['date', 'revenue_B', 'fcf_B', 'revenue_growth', 'fcf_growth', 'fcf_margin']].to_string(index=False))
    
    print(f"\n🎯 GROWTH RATE STATISTICS (Target Variables):")
    print(f"   revenue_growth: Mean={df['revenue_growth'].mean():.1f}%, Std={df['revenue_growth'].std():.1f}%")
    print(f"   fcf_growth:     Mean={df['fcf_growth'].mean():.1f}%, Std={df['fcf_growth'].std():.1f}%")
    
    print(f"\n💰 MARGIN STATISTICS:")
    print(f"   fcf_margin:       Mean={df['fcf_margin'].mean():.1f}%, Range=[{df['fcf_margin'].min():.1f}%, {df['fcf_margin'].max():.1f}%]")
    print(f"   operating_margin: Mean={df['operating_margin'].mean():.1f}%, Range=[{df['operating_margin'].min():.1f}%, {df['operating_margin'].max():.1f}%]")
    
    print(f"\n✅ DATA QUALITY:")
    missing = df[display_cols].isnull().sum()
    if missing.sum() == 0:
        print(f"   ✅ No missing values in key columns!")
    else:
        print(f"   Missing values:")
        for col in display_cols:
            if missing[col] > 0:
                print(f"      {col}: {missing[col]}")
    
    print(f"\n{'=' * 100}")
    print(f"✅ VALIDATION COMPLETE - Dataset is suitable for LSTM training!")
    print(f"{'=' * 100}")
    print(f"\n📋 KEY FEATURES FOR LSTM:")
    print(f"   ✓ Revenue, CapEx, D&A, Operating CF (real financial metrics)")
    print(f"   ✓ FCF = Operating CF - CapEx (not price-based proxy)")
    print(f"   ✓ Growth rates as target variables (revenue_growth, fcf_growth)")
    print(f"   ✓ Margins for quality assessment (fcf_margin, operating_margin)")
    print(f"   ✓ Normalized by Total Assets (revenue_per_asset, fcf_per_asset)")
    
    print(f"\n📝 NEXT STEP:")
    print(f"   Create script: scripts/build_enhanced_training_data.py")
    print(f"   - Loop through all {len(tickers_with_financials)} stocks")
    print(f"   - Apply same logic as above")
    print(f"   - Save to: data/processed/training/lstm_dcf_training_enhanced.csv")
    print(f"   - Expected output: ~8,000 records with 25-29 features")

except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    print(traceback.format_exc())


🔍 QUICK DATASET VALIDATION (AAPL Only)

✅ Loaded AAPL financial statements
   Income: 81 quarters
   Cashflow: 81 quarters
   Balance: 81 quarters

✅ COMBINED DATASET
   Shape: (81, 26)
   Date range: 2005-06-30 to 2025-06-30

📊 SAMPLE: Last 5 Quarters
      date  revenue_B  fcf_B  revenue_growth  fcf_growth  fcf_margin
2024-06-30     85.777 26.707       -5.483014   29.056731   31.135386
2024-09-30     94.930 23.903       10.670693  -10.499120   25.179606
2024-12-31    124.300 26.995       30.938586   12.935615   21.717619
2025-03-31     95.359 20.881      -23.283186  -22.648639   21.897251
2025-06-30     94.036 24.405       -1.387389   16.876586   25.952827

🎯 GROWTH RATE STATISTICS (Target Variables):
   revenue_growth: Mean=8.0%, Std=29.8%
   fcf_growth:     Mean=17.7%, Std=94.1%

💰 MARGIN STATISTICS:
   fcf_margin:       Mean=23.6%, Range=[-7.3%, 49.1%]
   operating_margin: Mean=26.4%, Range=[11.4%, 39.3%]

✅ DATA QUALITY:
   Missing values:
      revenue_growth: 1
      fcf_growth

## 📋 Summary & Next Steps

### ✅ What We Fixed

| Aspect | Old (❌ Wrong) | New (✅ Correct) |
|--------|---------------|-----------------|
| **Data Source** | Price-based (close, EPS) | Financial statements (Income, CF, BS) |
| **Key Features** | FCFF_proxy = close × EPS × 0.7 | Real FCF = Operating CF - CapEx |
| **Target Variable** | Absolute FCFF value | Growth rates (%) |
| **Feature Count** | 12 features | 29 features |
| **Margins** | None | Operating, Net, FCF, EBITDA margins |
| **Normalization** | MinMaxScaler on prices | Divide by Total Assets |

### 🎯 New Feature Set (29 features)

**Core Financial Metrics (9)**
- Revenue, CapEx, D&A, FCF, Operating CF, EBITDA, Total Assets, Net Income, Operating Income

**Profitability Margins (4)**
- Operating Margin, Net Margin, FCF Margin, EBITDA Margin

**Asset Efficiency (2)**
- Asset Turnover, ROA

**Normalized by Assets (5)**
- Revenue/Asset, CapEx/Asset, D&A/Asset, FCF/Asset, EBITDA/Asset

**Growth Rates - TARGET VARIABLES (7)**
- Revenue Growth (QoQ & YoY), FCF Growth (QoQ & YoY), CapEx Growth, D&A Growth, EBITDA Growth

### 📝 Implementation Steps

1. **Expand to All Stocks** (scripts/build_enhanced_training_data.py)
   ```python
   all_data = []
   for ticker in tickers_with_financials:
       result = build_proper_training_dataset(ticker)
       if result['success']:
           all_data.append(result['data'])
   
   training_df = pd.concat(all_data, ignore_index=True)
   training_df.to_csv('data/processed/training/lstm_dcf_training_enhanced.csv', index=False)
   ```

2. **Update LSTM Model** (src/models/deep_learning/lstm_dcf.py)
   - Input: 29 features (not 12)
   - Output: Growth rates (3 values: revenue, fcf, ebitda)
   - Use StandardScaler (not MinMaxScaler)

3. **Retrain Model** (scripts/train_lstm_dcf_enhanced.py)
   - Load new training data
   - Configure for growth rate prediction
   - Train with GPU acceleration (~10 minutes)

4. **Update Agents** (src/agents/enhanced_valuation_agent.py)
   - Use growth rate predictions in DCF
   - Apply predicted margins
   - Calculate fair value from forecasted FCF

### 🎓 Expected Results

- ✅ Realistic fair values ($300-700 for MSFT, not $12K)
- ✅ Growth rates between -20% to +30% (realistic range)
- ✅ Model learns business fundamentals (not price patterns)
- ✅ Margins provide context for quality assessment